In [ ]:
### Analysis of Joint-Profiled Single Cell (SC) DNA and Cell-Surface Protein Data
### All data was initially processed on an open-source pipeline: https://github.com/AbateLab/DAb-seq
### Resulting HDF5 files and H5AD files were then imported for downstream analyses

In [ ]:
from __future__ import division
import itertools
import math
import numpy as np
import pandas as pd
from collections import Counter
import seaborn as sns; sns.set(style="white", color_codes=True)
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as mcol
plt.rcParams['axes.unicode_minus'] = False
from sklearn.manifold import TSNE
from scipy.stats import spearmanr
from scipy.spatial.distance import pdist, squareform
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster, set_link_color_palette
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
import os
from IPython.display import display, HTML
from scipy.stats.mstats import gmean
from matplotlib.ticker import NullFormatter
from scipy.stats import binned_statistic
from IPython.display import display, HTML, Image
import h5py
import copy
import umap
matplotlib.rc('font', family='Helvetica')
import importlib
import re 
from matplotlib.pyplot import cm
import anndata as ad
import scanpy as sc
import sys
from scipy.signal import find_peaks
from scipy.optimize import curve_fit
from scipy.stats import norm
import scipy.stats as stats

In [ ]:
#Set thresholds for quality and total depth
min_quality = 30
min_total_depth = 10
min_alt_depth = 4

In [ ]:
#Set antibodies used in DAb-seq analysis
abs = ['CD10', 'CD117', 'CD11b', 'CD123', 'CD13', 'CD14', 'CD15', 'CD16','CD19', 'CD22', 'CD3', 'CD30', 
          'CD33', 'CD34', 'CD38', 'CD4', 'CD5', 'CD56', 'CD64', 'CD7', 'CD71','CD8']

In [1]:
#Load Genotype and Variant Information From Compressed H5 File
#The below script separates the h5 file into separate genotype and variant matrices

def load_genotypes(genotypes_path):

    with h5py.File(genotypes_path, 'r') as f:
         
        cell_barcodes = copy.deepcopy([c.decode('utf8').split('.')[0] for c in f['CELL_BARCODES']])
        variants = copy.deepcopy([v.decode('utf8') for v in f['VARIANTS']])
        
        genotypes = pd.DataFrame(np.transpose(f['GT']), index=cell_barcodes, columns=variants).sort_index()
        genotypes.index.name = 'cell_barcode'
        
        quality = pd.DataFrame(np.transpose(f['GQ']), index=cell_barcodes, columns=variants).sort_index()
        quality.index.name = 'cell_barcode'
        
        total_depth = pd.DataFrame(np.transpose(f['DP']), index=cell_barcodes, columns=variants).sort_index()
        total_depth.index.name = 'cell_barcode'
        
        alt_depth = pd.DataFrame(np.transpose(f['AD']), index=cell_barcodes, columns=variants).sort_index()
        alt_depth.index.name = 'cell_barcode'
                      
    return genotypes, quality, total_depth, alt_depth

def load_variants(variants_file_path):
    variant_info = pd.read_csv(variants_file_path, sep='\t', header=0, index_col='Name', low_memory=False)
    variant_info.index.name = 'variant' 
    
    return variant_info

In [ ]:
#Filter variant calls from genotyping data based on pre-set metrics

def filter_variants(genotypes,
                    alt_depth,
                    total_depth,
                    quality,
                    min_alt_depth,
                    min_total_depth,
                    min_quality):

    genotypes[total_depth < min_total_depth] = 3
    genotypes[((genotypes == 1) | (genotypes == 2)) & (alt_depth < min_alt_depth)] = 3
    genotypes[quality < min_quality] = 3
    genotypes[genotypes.isnull()] = 3
    
    # Quality = 0 for FLT3-ITD implies WT
    genotypes_flt3 = genotypes.loc[:, genotypes.filter(like='FLT3-ITD', axis=1).columns]
    quality_flt3 = quality.loc[:, quality.filter(like='FLT3-ITD', axis=1).columns]
    
    genotypes_flt3[quality_flt3 == 0] = 0
    genotypes.loc[:, genotypes.filter(like='FLT3-ITD', axis=1).columns] = genotypes_flt3

    return genotypes

In [1]:
# Set experimental metadata and filepaths, load data
genotypes_path = 'MPAL1.genotypes.hdf5'
variant_info_path = 'MPAL1.variants.tsv'
antibodies_path = 'MPAL1_Ab.adata.h5ad'


# Load Genotyping Data and Variant Info
genotypes, quality, total_depth, alt_depth = load_genotypes(genotypes_path)
variant_info = load_variants(variant_info_path)


# Apply Quality and Depth Filters to All Genotypes
genotypes = filter_variants(genotypes, alt_depth, total_depth, quality, min_alt_depth, min_total_depth, min_quality)


# Load and Format Antibody File
adata = ad.read_h5ad(antibodies_path)
num = file.split('_Ab.adata')[0][-1]
adata.obs_names = [bc+'-'+num for bc in adata.obs_names]

with open('MPAL1.cells.tsv') as f:
    geno = [l.strip().split('\t') for l in f]
genes = geno[0][1:]
barcodes = [l[0].replace('M', '-') for l in geno[1:]]
geno_counts = np.array([l[1:] for l in geno[1:]]).astype(int)

adata_g = ad.AnnData(X=geno_counts)
adata_g.obs_names = barcodes
adata_g.var_names = genes


#Identify Joint Barcodes between Genotypes File and Ab File
intersect = np.sort(list(set(barcodes) & set(adata.obs_names)))
adata = adata[intersect]
adata_g = adata_g[intersect]
len(intersect)


#Filtering Genotype Data based on pre-set metrics
#In the below analysis, we only consider variants for which > 0.001 reads are mutated for a single cell
min_fraction_mutated = 0.001
min_fraction_genotyped = 0.50

number_genotyped = (genotypes != 3).sum(axis=0)
fraction_genotyped = pd.DataFrame(number_genotyped / genotypes.shape[0], columns=['fraction_genotyped'])
number_mutated = ((genotypes == 1) | (genotypes == 2)).sum(axis=0)
fraction_mutated = pd.DataFrame(number_mutated.divide(number_genotyped), columns=['fraction_mutated']).dropna()


#Filter Genotype Data based on ClinVar annotation
#Remove non-intronic variants, remove synonymous variants, include only pathogenic variants (also manually reviewed)
variant_info_1 = variant_info[(variant_info["SnpEff_HGVS_p"].notnull())]
genotypes = genotypes[list(variant_info_1.index)]
variant_info_2 = variant_info[variant_info["SnpEff_Annotation"] != "synonymous_variant"]
genotypes = genotypes[list(variant_info_2.index)]
variant_info_3 = variant_info[variant_info["SnpEff_Annotation_Impact"] == 'HIGH']
genotypes = genotypes[list(variant_info_3.index)]


#Call Genetic Clones
clonal_set = list(genotypes.columns[np.apply_along_axis(lambda x: sum(x) > 2, 0, final_NGTs.values[:, 1:])])
clone_cutoff = 10

def assign_clones(c):
    genotypes_clones = c.iloc[:, 1:].sum().sort_values(ascending=False).index
    clone_colnames = ['Clone'] + list(genotypes_clones)
    clone_df = pd.concat([y['Cell'], c.iloc[:, 1:].loc[:, genotypes_clones]], axis=1)
    clone_df['Clone'] = clone_df[genotypes_clones].apply(lambda x: '_'.join(x.index[x.values.argsort()[::-1]] >= clone_cutoff), axis=1)
    clone_df = clone_df.loc[:, clone_colnames]
    return clone_df

clones = [assign_clones(c) for c in clonal_set]


#CLR Transformation of Antibody Data
def clr_transform(andat_raw, add_pseudocount:bool=False, features:list=[]):
        if add_pseudocount:
            clr_data = np.log((1+andat_raw[:,features].X)/gmean(andat_raw[:,features].X+1, axis=1).reshape(-1,1))
        else:
            clr_data = np.log(andat_raw[:,features].X/gmean(andat_raw[:,features].X, axis=1).reshape(-1,1))
        return alr_data, clr_data, ilr_data
            
adata.var['Ab_umi'] = adata.X.sum(axis=0)
adata.obs['gene_count'] = adata_g.X.sum(axis=1)
adata_g.var['amplicon_count'] = adata_g.X.sum(axis=0)
adata_g.obs['umi_counts'] = adata.obs['umi_counts']
adata_g.obs['raw_counts'] = adata.obs['raw_counts']
adata_ab = ad.AnnData(adata[:,abs].X, obs=adata[:,abs].obs, var=adata[:,signal].var)

ab_clr = clr_transform(adata_ab, ['IgG_count','umi_counts','raw_counts','gene_count'], 
                                         components=2, antibodies=signal)

#Generate Antibody-Derived UMAP
trans = umap.UMAP(n_neighbors=50, random_state=42).fit(ab_clr)
trans_results = trans.embedding_

plt.xlabel('UMAP 1', fontsize=12, labelpad=12)
plt.ylabel('UMAP 2', fontsize=12, labelpad=12)
plt.scatter(x=trans_results[:, 0],
            y=trans_results[:, 1],
            s=10)

#Overlay individual antibodies on the antibody-derived UMAP
fig = plt.figure(figsize=(8,8))
for i in range(len(ab_clr.columnes)):
    ax = fig.add_subplot(5, 5, i + 1)
    ax.scatter(trans_results[:, 0],
               trans_results[:, 1],
               alpha=0.8,
               c=ab_clr[ab_clr_columns[i]],
               cmap='magma',
               s=1)
    plt.title(ab_clr.columns[i].split('_')[0], fontsize=11)
    plt.xticks([])
    plt.yticks([])
plt.tight_layout()

#Overlay mutations of interest on antibody-derived UMAP (Figure 2C, 2G)
plt.xticks([])
plt.yticks([])
clones =[] #set clone of interest here

hue_map = {0: 'Wildtype',
           1: 'Het. Mutant',
           2: 'Hom. Mutant',
           3: 'No Call'}
    
color_map = {'Wildtype': 'lightgrey',
             'Het. Mutant': 'lightblue',
             'Hom. Mutatant': 'darkblue'}

sns.scatterplot(x=trans_results[:, 0],
                y=trans_results[:, 1], 
                hue=[hue_map[g] for g in genotypes[clone]],
                palette=color_map,
                hue_order=['Het. Mutant','Hom. Mutant','Wildtype'],
                edgecolor=None)

#Generate split-violin plots comparing mutant vs non-mutant populations (Figure 2D, 2H)
antibody = [] #set antibody of interest
sns.violinplot(data=ab_clr, x='all', y=ab_clr.columns[antibody], hue=clones,
               split=True, palette=['lightgrey','red'])
    
#Define Immunophenotypic Subpopulations via Hierarchical Clustering and overlay on antibody-derived UMAP (Figure 2D, 2H)
Z = linkage(trans_results, 'centroid')
max_d = 2   

dendrogram(Z, leaf_rotation=90.,leaf_font_size=8.,truncate_mode='level', p=4,max_d=max_d)
plt.axhline(y=max_d, c='white', lw=1.5, linestyle='dashed') 
ab_clusters = fcluster(Z, max_d, criterion='distance')  

ab_color = dict(zip(sorted(list(set(ab_clusters))),
                    [plt.cm.get_cmap('jet')(x) for x in np.linspace(0, 1, len(set(ab_clusters)))]))
c = [ab_color[c] for c in ab_clusters]
plt.scatter(trans_results[:, 0],
            trans_results[:, 1], 
            c=c,s=5,edgecolor=None)

#All patients loaded and pre-processed, as outlined above
#Calculate T-statistic of antibody signal between mutant vs non-mutant populations (Figure 2B)
#For this figure, all patients were concatenated into a single dataframe, "patients" where 0 is WT and 1 is mutant
ts = pd.DataFrame(index = Abs)

for p in patients:
    t = stats.ttest_ind(p[abs_clr][0],p[abs_clr][1])
    ts[p]=t[0]

sns.clustermap(ts, cmap= c, row_cluster = True, col_cluster = False, metric = 'centroid')

#Phylogeny Analysis (Figure 3)
#Separate models were constructed for individual patients to calculate t-statistic
import statsmodels.formula.api as smf

for a in abs_clr.columns:
    dat_ab = pd.DataFrame({abs_clr: dat[a], 'clones': pd.Categorical(dat['clones'])})
    dat_ab.columns = [a, 'clones']
    dat_ab['clones'] = dat_ab_1['clones'].cat.codes
    dat_ab['clones'] = dat_ab_1['Clone'].replace(to_replace={4: np.nan})
    dat_ab = pd.get_dummies(dat_ab_1, columns=['clones'], prefix='clones', prefix_sep='_', drop_first=True)
    dat_ab.columns = [col.replace('.', '_') for col in dat_ab_1.columns]

    fit = smf.ols(formula='a ~ clones', data=dat_ab_1).fit()
    t_statistic = fit.summary().tables[1][:,2]
    p = fit.summary().tables[1][:,3]
    
#All models then concatenated into single dataframe, "phylogeny" (Figure 3A)
phylogeny = phylogeny.sort_values(['t_statistic'], ascending = False)
sns.barplot(data=phylogeny,y='t_statistic',x='ab_clr',palette='Spectral',ci='sd')
sns.stripplot(data=phylogeny,y='t_statistic',x='ab_clr',color='black',edgecolor='gray')
plt.ylabel('Maximum t-statistic for Each Patient')
plt.xlabel('Cell-Surface Protein')

#Individual patient violinplots (Figure 3B)
top_abs = phylogeny.sort_values(['t_statistic'][:,0:4]
                                
for i in range(len(top_abs.columns)):
    if p <= 0.05*phylogeny.shape(): 
        ax = fig.add_subplot(5, 5, i + 1)
        sns.violinplot(x = "clone",y = top_abs.columns[i],data = phylogeny, palette="red")
        plt.xticks([]), plt.xlabel(""), plt.legend([],[], frameon=False)
    elif p <= -0.05*phylogeny.shape():
        ax = fig.add_subplot(5, 5, i + 1)
        sns.violinplot(x = "clone",y = top_abs.columns[i],data = phylogeny, palette="turquoise")
        plt.xticks([]), plt.xlabel(""), plt.legend([],[], frameon=False)
    else: 
        ax = fig.add_subplot(5, 5, i + 1)
        sns.violinplot(x = "clone",y = top_abs.columns[i],data = phylogeny, palette="grey")
        plt.xticks([]), plt.xlabel(""), plt.legend([],[], frameon=False)
                                
plt.tight_layout(pad=0)